In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from datetime import date
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn import preprocessing
from sklearn.metrics import accuracy_score, f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from imblearn.ensemble import EasyEnsembleClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.externals import joblib
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.under_sampling import TomekLinks
from imblearn.under_sampling import NearMiss
from imblearn.over_sampling import SMOTE, BorderlineSMOTE
from sklearn.model_selection import cross_validate
from sklearn import datasets,model_selection,metrics,neural_network,preprocessing
from sklearn.utils import shuffle

from google.colab import drive

import warnings
warnings.filterwarnings("ignore")
import math
import re
import datetime

In [0]:
def user_id_to_int(df):
  df['user_id'] = df['user_id'].apply(lambda x: int(x.split("_")[1]))
  return df


def add_inbound_outbound(users,users_trans):
  inbound_list = [0] * users.shape[0]
  outbound_list = [0] * users.shape[0]
  inbound_outbound_ratio_list  = [0] * users.shape[0]

  for idx_users_trans, users_trans_row in users_trans.iterrows():
    if users_trans_row['direction'] == 'INBOUND':
      inbound_list[users_trans_row['user_id']] = inbound_list[users_trans_row['user_id']] + 1
    elif users_trans_row['direction'] == 'OUTBOUND':
      outbound_list[users_trans_row['user_id']] = outbound_list[users_trans_row['user_id']] + 1

  for i in range(len(users)):
    inbound_outbound_total = inbound_list[i] + outbound_list[i]
    if (inbound_outbound_total == 0):
      inbound_outbound_ratio_list[i] = 0
    else:  
      inbound_outbound_ratio_list[i] = inbound_list[i] / inbound_outbound_total

  
  inbound_ser = pd.Series(inbound_list)
  outbound_ser = pd.Series(outbound_list)
  inbound_outbound_ratio_ser = pd.Series(inbound_outbound_ratio_list)
  users['inbound_transactions'] = inbound_ser
  users['outbound_transactions'] = outbound_ser
  users['inbound_outbound_ratio'] = inbound_outbound_ratio_ser
  return users


def add_completed_to_failed_trans_ratio(users,users_trans):
  completed_list = [0] * users.shape[0]
  failed_list = [0] * users.shape[0]
  completed_failed_ratio_list  = [0] * users.shape[0]

  for idx_users_trans, users_trans_row in users_trans.iterrows():
    if users_trans_row['transactions_state'] == 'COMPLETED':
      completed_list[users_trans_row['user_id']] = completed_list[users_trans_row['user_id']] + 1
    elif users_trans_row['transactions_state'] == 'FAILED':
      failed_list[users_trans_row['user_id']] = failed_list[users_trans_row['user_id']] + 1

  for i in range(len(users)):
    completed_failed_total = completed_list[i] + failed_list[i]
    if (completed_failed_total == 0):
      completed_failed_ratio_list[i] = 0
    else:  
      completed_failed_ratio_list[i] = completed_list[i] / completed_failed_total

  
  completed_ser = pd.Series(completed_list)
  failed_ser = pd.Series(failed_list)
  completed_failed_ratio_ser = pd.Series(completed_failed_ratio_list)
  users['completed_transactions'] = completed_ser
  users['failed_transactions'] = failed_ser
  users['completed_failed_ratio'] = completed_failed_ratio_ser
  return users


def cardholder_fillna(df):
  df['ea_cardholderpresence'] = df['ea_cardholderpresence'].fillna('UNKNOWN')

  df['ea_cardholderpresence'] = df['ea_cardholderpresence'].apply(cardholder_pres_fillna)
  return df


def remove_outliers(series):
  x = series
  UPPERBOUND, LOWERBOUND = np.percentile(x, [1,99])
  y = np.clip(x, UPPERBOUND, LOWERBOUND)
  return pd.Series(y)


def normalize(series, scaler):
  column_as_array = series.to_numpy().reshape(-1,1)
  scaled = scaler.transform(column_as_array)
  scaled_df = pd.DataFrame(scaled)
  return scaled_df[0]


def standardize(series, scaler):
  column_as_array = series.to_numpy().reshape(-1,1)
  scaled = scaler.transform(column_as_array)
  scaled_df = pd.DataFrame(scaled)
  return scaled_df[0]


def cardholder_fillna(df):
  df['ea_cardholderpresence'] = df['ea_cardholderpresence'].fillna('UNKNOWN')

  df['ea_cardholderpresence'] = df['ea_cardholderpresence'].apply(cardholder_pres_fillna)
  return df


def cardholder_pres_fillna(presence):
  
  if presence == 'UNKNOWN':
    return 'FALSE'
  else:
    return presence


def add_months_subscribed(users):
  now = datetime.datetime.now()
  months_subscribed_list = [0] * users.shape[0]

  for idx_users, users_row in users.iterrows():
    date_split = users_row['created_date'].split("-")
    year_created = int(date_split[0])
    month_created = int(date_split[1])
    if now.year == year_created:
      total_months = now.month - month_created
    else:
      diff = now.year - year_created  
      total_months = now.month + ((diff * 12) - month_created)
    
    months_subscribed_list[idx_users] = total_months

  months_subscribed_ser = pd.Series(months_subscribed_list)
  users['months_subscribed'] = months_subscribed_ser

  return users


def mapping_data(users):  
  mapping_dict_plan = {'STANDARD': 0, 'SILVER': 1, 'GOLD': 1}
  users['plan'] = users['plan'].map(mapping_dict_plan)
  return users


def extract_trans_features(users, transactions):

    user_trans = pd.merge(transactions, users,on='user_id',how='left')

    inbound_per_user = pd.DataFrame(transactions[transactions['direction']== 'INBOUND'].groupby(['user_id'])['direction'].size().reset_index(name='inbound_transactions'))
    users = pd.merge(inbound_per_user, users, on='user_id', how='outer')
    
    outbound_per_user = pd.DataFrame(transactions[transactions['direction']== 'OUTBOUND'].groupby(['user_id'])['direction'].size().reset_index(name='outbound_transactions'))
    users = pd.merge(outbound_per_user, users, on='user_id', how='outer')



    completed_per_user = pd.DataFrame(transactions[transactions['transactions_state']== 'COMPLETED'].groupby(['user_id'])['transactions_state'].size().reset_index(name='completed_transactions'))
    users = pd.merge(completed_per_user, users, on='user_id', how='outer')

    declined_per_user = pd.DataFrame(transactions[transactions['transactions_state']== 'DECLINED'].groupby(['user_id'])['transactions_state'].size().reset_index(name='declined_transactions'))
    users = pd.merge(declined_per_user, users, on='user_id', how='outer')

    reverted_per_user = pd.DataFrame(transactions[transactions['transactions_state']== 'REVERTED'].groupby(['user_id'])['transactions_state'].size().reset_index(name='reverted_transactions'))
    users = pd.merge(reverted_per_user, users, on='user_id', how='outer')

    failed_per_user = pd.DataFrame(transactions[transactions['transactions_state']== 'FAILED'].groupby(['user_id'])['transactions_state'].size().reset_index(name='failed_transactions'))
    users = pd.merge(failed_per_user, users, on='user_id', how='outer')

    pending_per_user = pd.DataFrame(transactions[transactions['transactions_state']== 'PENDING'].groupby(['user_id'])['transactions_state'].size().reset_index(name='pending_transactions'))
    users = pd.merge(pending_per_user, users, on='user_id', how='outer')

    cancelled_per_user = pd.DataFrame(transactions[transactions['transactions_state']== 'CANCELLED'].groupby(['user_id'])['transactions_state'].size().reset_index(name='cancelled_transactions'))
    users = pd.merge(cancelled_per_user, users, on='user_id', how='outer')

    #number of transactions per user
    trans_per_user = pd.DataFrame(transactions.groupby(['user_id'])['transactions_state'].size().reset_index(name='TransPerUser'))

   
    users['inbound_outbound_ratio'] = users['inbound_transactions'] / (users['inbound_transactions'] + users['outbound_transactions'])

    users['completed_transactions'] = users['completed_transactions'] / trans_per_user['TransPerUser']
    users['declined_transactions'] = users['declined_transactions'] / trans_per_user['TransPerUser']
    users['reverted_transactions'] = users['reverted_transactions'] / trans_per_user['TransPerUser']
    users['failed_transactions'] = users['failed_transactions'] / trans_per_user['TransPerUser']
    users['pending_transactions'] = users['pending_transactions'] / trans_per_user['TransPerUser']
    users['cancelled_transactions'] = users['cancelled_transactions'] / trans_per_user['TransPerUser']
    
    return users.fillna(0)


#Giorgos
#adjusted net annual income per capita
dict2018 = {'GB' : 35837, 'PL':12907,'FR':34837,'IE':43843,'RO':9561,'ES':25686,
            'LT':16125,'PT':18799,'MT': 17156 ,'DE':40265,'CH':65578,'CZ':17146,'IT':28806,
            'GR':17147,'CY':26328,'LV':13726,'NL' : 44668,'HU':12896,'BE':38814,'SE':46002,'BG':8073,
            'DK':51867,'NO':66019,'SI':20691,'AT':42074,'SK':15059,
            'HR':12586,'FI':41120,'EE':18993,'LU':60159,'IS' : 58011,'LI': 59872,'AU' : 43346}
 
#groups =  0-20k,20k-40k,40k-60k,above 60k
 
def group_countries(adict):
    for key,value in adict.items():
        if value < 20000:
            adict[key] =  1
        elif value >=20000 and value <40000:
            adict[key] = 2
        elif value < 60000 and  value >= 40000:
            adict[key] = 3
        else:
            adict[key] =4
    return adict


def extract_nots_features(users, notifications):
    notifications = extract_date_features(notifications)
    notifications = pd.merge(notifications,users,on='user_id',how='left')

    #number of notifications per user
    nots_per_user = pd.DataFrame(notifications.groupby(['user_id'])['channel'].size().reset_index(name='NotsPerUser'))
    users = pd.merge(nots_per_user,users,on='user_id',how='outer')
    
    #number of emails per user
    emails_per_user = pd.DataFrame(notifications[notifications['channel']== 'EMAIL'].groupby(['user_id'])['channel'].size().reset_index(name='EmailsPerUser'))
    users = pd.merge(emails_per_user,users,on='user_id',how='outer')

    
    #number of push per user
    push_per_user = pd.DataFrame(notifications[notifications['channel'] == 'PUSH'].groupby('user_id')['channel'].size().reset_index(name='PushPerUser'))
    users = pd.merge(push_per_user,users,on='user_id',how='outer')
    
    #number of SMS per user
    sms_per_user = pd.DataFrame(notifications[notifications['channel'] == 'SMS'].groupby('user_id')['channel'].size().reset_index(name='SmsPerUser'))
    users = pd.merge(sms_per_user,users,on='user_id',how='outer')
   
    users['AvgNotsPerMonth'] = users['NotsPerUser'] / users['months_subscribed']
    users['AvgEmailsPerMonth'] = users['EmailsPerUser'] / users['months_subscribed']
    users['AvgPushPerMonth'] = users['PushPerUser'] / users['months_subscribed']
    users['AvgSmsPerMonth'] = users['SmsPerUser'] / users['months_subscribed']
    
    return users.fillna(0)


def clean_string(astring):
    #convert to upper and replace empty space
    astring = astring.replace(" ","").upper()
    #remove numbers
    clean = ''.join([i for i in astring if not i.isdigit()])
    #remove special characters
    clean = re.sub(r"^\W+", "", clean)
    #if empty rename to Other
    clean.strip()
    if len(clean) < 3 or clean == "" : clean = 'OTHER'
    #check for buchurest and prague
    if(clean == 'BUCURESTI' or clean == 'BUCUREȘTI'): clean = 'BUCHAREST'
    if(clean == 'PRAHA'): clean = 'PRAGUE'
    if(clean == 'WARSZAWA'): clean = 'WARSAW'
    if(clean == 'CO.DOUBLIN'): clean = 'DUBLIN'
    if(clean == 'LISBOA'): clean = 'LISBON'
    if(clean == 'MILANO'): clean = 'MILAN'
    if(clean == 'RĪGA'): clean = 'RIGA'
    if(clean == 'BRUXELLES'): clean = 'BRUSSELS'
    return clean


#Map countries to other countries
c2c_dict = {'MQ' : 'FR','RE' : 'FR','GP' : 'FR','GI' : 'ES','GG' : 'GB','JE' : 'GB','IM' : 'IE'}            
def c2c(country):
    if country in c2c_dict.keys():
        return c2c_dict.get(country)
    return country


def extract_date_features(dataframe):
    dataframe['created_date'] = [datetime.datetime.strptime(str(date_time_str), '%Y-%m-%d %H:%M:%S.%f') for date_time_str in dataframe['created_date']]
    dataframe['part_of_day'] = (dataframe['created_date'].dt.hour %24 + 4) //4
    '''dataframe['part_of_day'].replace({1: 'Late Night',
                      2: 'Early Morning',
                      3: 'Morning',
                      4: 'Noon',
                      5: 'Evening',
                      6: 'Night'},inplace=True)'''
    dataframe['part_of_week'] = dataframe['created_date'].dt.weekday
    '''dataframe['part_of_week'].replace({0:'Weekday',
                                      1:'Weekday',
                                      2:'Weekday',
                                      3:'Weekday',
                                      4:'Weekday',
                                      5:'Weekend',
                                      6:'Weekend'},inplace=True)'''
    dataframe['Half_of_month'] = dataframe['created_date'].dt.day.apply(half_of_month)
    dataframe['Month'] = dataframe['created_date'].dt.month
    dataframe['Epoch'] = dataframe['created_date'].dt.month.apply(epoch_of_year).map({'Winter':1,'Spring':2,'Summer':3,'Autumn':4})
    dataframe['Year'] = dataframe['created_date'].dt.year

    return dataframe


def epoch_of_year(month):
    if month in [12,1,2]:
        return 'Winter'
    elif month in [3,4,5]:
        return 'Spring'
    elif month in [6,7,8]:
        return 'Summer'
    else:
        return 'Autumn'


def half_of_month(day):
    if day <= 15 : return 1
    return 2

In [18]:
drive.mount('/content/drive')
project_dir = '/content/drive/My Drive/Colab Notebooks/Data Science-Project Future/pf-ds-thes-team2/'

devices_file = project_dir + 'test/devices_test.csv'
users_file = project_dir + 'test/users_test.csv'
trans_file = project_dir + 'test/transactions_test.csv'
notif_file = project_dir + 'test/notifications_test.csv'
country_codes_file = project_dir + 'auxiliary_data/country_codes.csv'

users_raw = pd.read_csv(users_file)
users = users_raw
devices = pd.read_csv(devices_file)
transactions = pd.read_csv(trans_file)
notifications = pd.read_csv(notif_file)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
model = joblib.load(project_dir + 'auxiliary_data/ensemble_classifier.pkl')

min_max_scaler_inbound_trans = joblib.load(project_dir + 'auxiliary_data/scalers/MinMax_inbound_transactions.save')
min_max_scaler_NotsPerUser = joblib.load(project_dir + 'auxiliary_data/scalers/MinMax_NotsPerUser.save')
min_max_scaler_outbound_trans = joblib.load(project_dir + 'auxiliary_data/scalers/MinMax_outbound_transactions.save')
stand_scaler_AvgNotsPerMonth = joblib.load(project_dir + 'auxiliary_data/scalers/Stand_AvgNotsPerMonth.save')
stand_scaler_AvgPushPerMonth = joblib.load(project_dir + 'auxiliary_data/scalers/Stand_AvgPushPerMonth.save')
stand_scaler_inbound_outbound_ratio = joblib.load(project_dir + 'auxiliary_data/scalers/Stand_inbound_outbound_ratio.save')


In [0]:
users = user_id_to_int(users)
devices = user_id_to_int(devices)
transactions = user_id_to_int(transactions)
notifications = user_id_to_int(notifications)
transactions = cardholder_fillna(transactions)

users_trans = users.merge(transactions, on='user_id', how='left')

In [0]:
users = add_months_subscribed(users)
users = extract_trans_features(users, transactions)

cols_to_drop = ['attributes_notifications_marketing_push', 'attributes_notifications_marketing_email']
users = users.drop(columns = cols_to_drop)

users = users.reset_index(drop=True)

In [0]:
users['country'] = users['country'].apply(c2c)
users['country'] = users['country'].map(group_countries(dict2018))

users = extract_nots_features(users, notifications)

In [0]:
users = users[['inbound_transactions', 'outbound_transactions', 'user_settings_crypto_unlocked', 'country', 'NotsPerUser', 'AvgNotsPerMonth', 'AvgPushPerMonth', 'inbound_outbound_ratio']]

In [0]:
users['inbound_transactions'] = remove_outliers(users['inbound_transactions'])
users['inbound_transactions'] = normalize(users['inbound_transactions'], min_max_scaler_inbound_trans)

users['NotsPerUser'] = remove_outliers(users['NotsPerUser'])
users['NotsPerUser'] = normalize(users['NotsPerUser'], min_max_scaler_NotsPerUser)

users['outbound_transactions'] = remove_outliers(users['outbound_transactions'])
users['outbound_transactions'] = normalize(users['outbound_transactions'], min_max_scaler_outbound_trans)

users['AvgNotsPerMonth'] = standardize(users['AvgNotsPerMonth'], stand_scaler_AvgNotsPerMonth)
users['AvgPushPerMonth'] = standardize(users['AvgPushPerMonth'], stand_scaler_AvgPushPerMonth)
users['inbound_outbound_ratio'] = standardize(users['inbound_outbound_ratio'], stand_scaler_inbound_outbound_ratio)

In [0]:
X = users

In [0]:
predictions = model.predict(X)

In [0]:
plan_ser = pd.Series(predictions)
user_id_ser = users_raw['user_id']

frame = { 'user_id': user_id_ser, 'plan': plan_ser } 
  
predictions_df = pd.DataFrame(frame) 

predictions_df.to_csv(project_dir + 'predictions/test_preds.csv', index = False)